In [25]:
import pandas as pd
import numpy as np
import random
import json

In [26]:
source = "../data/normalized_new_zeland.csv"
sink = "../generated"
synonims_path = "../data/synonims.json"

In [86]:
def load_synonims(path):
    with open(path, "r") as f:
        data = json.load(f)
    return {
        key: value + [key] for key, value in data.items()
    }

In [87]:
def sample(array):
    return random.sample(array.tolist(), random.randint(1, array.size))

In [92]:
def generate_dataframe(original, synonims, path):
    cols = sample(original.columns)
    index = sample(original.index)
    new = original[cols].loc[index]
    rename = {
        c:random.choice(synonims[c]) for c in cols
    }
    new = new.rename(index=str, columns=rename)
    
    return new

In [93]:
synonims = load_synonims(synonims_path)

In [94]:
dataframe = pd.read_csv(source, index_col=0)

In [95]:
generate_dataframe(dataframe, synonims, "")

,total monounsaturated fats,vitamin d,lipid,vactochrome,total cobalamin,polyunsaturated lipid,phosphorus p
807,"0,3",0,1,"0,26","0,35",trace,55
1113,"0,2",11,"1,3","0,02","0,63","0,4",260
993,"3,3",0,10,"0,04",0,"0,7",23
1568,"11,2",0,25,"0,12",1,"1,9",130
2251,"0,3",0,"0,8","0,06",0,"0,5",50
1357,"3,1","0,3","9,2","0,09","1,4","0,3",140
239,0,0,0,0,0,0,0
495,"0,5",0,"1,1","1,4",0,"0,2",99
485,"0,8",0,"2,6","3,4",0,"0,8",420
2097,trace,0,"0,2","0,13",0,"0,1",46
